In [1]:
import numpy as np
import torch

from train import TrainerEGBAD
from preprocess import get_mnist

In [2]:
class Args:
    num_epochs=100
    lr=0.00001
    latent_dim=200
    anormal_class=0
    batch_size=100
    
    
args = Args()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = get_mnist(args)

egbad = TrainerEGBAD(args, data, device)

In [ ]:
egbad.train()

43261/43261: [===============================>] - ETA 0.4sss
Training... Epoch: 0, Discrimiantor Loss: 1.181, Generator Loss: 1.665
43261/43261: [===============================>] - ETA 0.1ss
Training... Epoch: 1, Discrimiantor Loss: 0.285, Generator Loss: 4.903
43261/43261: [===============================>] - ETA 0.0ss
Training... Epoch: 2, Discrimiantor Loss: 0.046, Generator Loss: 8.626
43261/43261: [===============================>] - ETA 0.1ss
Training... Epoch: 3, Discrimiantor Loss: 0.017, Generator Loss: 10.875
43261/43261: [===============================>] - ETA 0.0ss
Training... Epoch: 4, Discrimiantor Loss: 0.010, Generator Loss: 12.255
43261/43261: [===============================>] - ETA 0.0ss
Training... Epoch: 5, Discrimiantor Loss: 0.006, Generator Loss: 13.375
43261/43261: [===============================>] - ETA 0.0ss
Training... Epoch: 6, Discrimiantor Loss: 0.004, Generator Loss: 14.454
43261/43261: [===============================>] - ETA 0.0ss
Training... Epoch:

43261/43261: [===============================>] - ETA 0.1ss
Training... Epoch: 62, Discrimiantor Loss: 0.001, Generator Loss: 30.215
43261/43261: [===============================>] - ETA 0.0ss
Training... Epoch: 63, Discrimiantor Loss: 0.002, Generator Loss: 30.163
43261/43261: [===============================>] - ETA 0.0ss
Training... Epoch: 64, Discrimiantor Loss: 0.001, Generator Loss: 30.332
43261/43261: [===============================>] - ETA 0.0ss
Training... Epoch: 65, Discrimiantor Loss: 0.001, Generator Loss: 30.366
43261/43261: [===============================>] - ETA 0.0ss
Training... Epoch: 66, Discrimiantor Loss: 0.003, Generator Loss: 31.260
43261/43261: [===============================>] - ETA 0.0ss
Training... Epoch: 67, Discrimiantor Loss: 0.002, Generator Loss: 31.176
43261/43261: [===============================>] - ETA 0.0ss
Training... Epoch: 68, Discrimiantor Loss: 0.001, Generator Loss: 30.630
43261/43261: [===============================>] - ETA 0.0ss
Training.

In [ ]:
from torch.autograd import Variable
import torch.nn.functional as F


alpha = 0.9
labels = []
scores = []

egbad.G.eval()
egbad.D.eval()

with torch.no_grad():
    for img, label in data[1]:
        img = img.float().to(device)
        img_hat = egbad.G(egbad.E(img))
        score_g = torch.sum(torch.abs(img - img_hat), dim=(1,2,3))
        
        y_true = Variable(torch.ones((img.size(0), 1)).to(device))
        y_pred = egbad.D(img, egbad.E(img)).view(-1)
        score_d = F.binary_cross_entropy(y_pred, y_true)
        
        score = alpha * score_g + (1-alpha)* score_d
        scores.append(score.cpu())
        labels.append(label.cpu())

In [ ]:
scores = torch.cat(scores, dim=0)
labels = torch.cat(labels, dim=0)

In [ ]:
from sklearn.metrics import precision_recall_curve, roc_auc_score, auc

precision, recall, thresholds = precision_recall_curve(labels, scores)

print('ROC AUC score: {:.2f}'.format(roc_auc_score(labels, scores)*100))
print('PR AUC score: {:.2f}'.format(auc(recall, precision)*100))

In [ ]:
import matplotlib.pyplot as plt

plt.hist(scores[labels==0], color='r')
plt.hist(scores[labels==1], color='b')
plt.show()